# Descritores Locais: SIFT, ORB e Matching

**Nota:** Este notebook foca na descrição local de imagens, permitindo o reconhecimento de objetos sob oclusão, rotação e mudanças de escala.

## 1) O Pipeline de Descritores Locais

O processo padrão para trabalhar com features locais envolve:
1.  **Detecção:** Encontrar pontos de interesse (keypoints) distintos.
2.  **Descrição:** Extrair um vetor numérico (fingerprint) da vizinhança de cada ponto.
3.  **Matching:** Comparar vetores entre imagens para achar correspondências.
4.  **Transformação:** Usar os matches para geometria (homografia) ou estatística (BoVW).

> **Conceito:** Após a extração, a imagem original é frequentemente descartada, e trabalhamos apenas com a nuvem de vetores descritores.

## 2) SIFT (Scale-Invariant Feature Transform)

O SIFT (Lowe, 2004) é o algoritmo clássico que resolve a invariância a escala e rotação.

### Funcionamento Resumido:
- **Detecção (Scale Space):** Usa Diferença de Gaussianas (DoG) para achar pontos estáveis em várias escalas.
- **Descrição (HOG Local):** Divide a vizinhança 16x16 em 4x4 sub-blocos. Em cada sub-bloco, calcula um histograma de 8 orientações.
- **Resultado:** Um vetor de $4 \times 4 \times 8 = 128$ dimensões.

O SIFT é robusto, mas computacionalmente intenso.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import data

# Setup: Imagem original e uma versão transformada (Simulando outra vista)
img1_cl = data.camera()
rows, cols = img1_cl.shape
# Rotação de 30 graus e escala de 1.1x
M = cv2.getRotationMatrix2D((cols/2, rows/2), 30, 1.1)
img2_cl = cv2.warpAffine(img1_cl, M, (cols, rows))

# 1. Instanciar SIFT (com fallback para ORB)
try:
    detector = cv2.SIFT_create()
    norm = cv2.NORM_L2
    is_orb = False
except AttributeError:
    print("SIFT indisponível, usando ORB.")
    detector = cv2.ORB_create()
    norm = cv2.NORM_HAMMING
    is_orb = True

# 2. Detectar e Descrever
kp1, des1 = detector.detectAndCompute(img1_cl, None)
kp2, des2 = detector.detectAndCompute(img2_cl, None)

# Visualizar Keypoints
out_img = cv2.drawKeypoints(img1_cl, kp1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.figure(figsize=(6,6))
plt.imshow(out_img)
plt.title(f"Detecção ({'ORB' if is_orb else 'SIFT'}): {len(kp1)} pontos")
plt.axis('off')
plt.show()

### Matching com Ratio Test
Para conectar os pontos, usamos um Matcher. O **Ratio Test de Lowe** descarta matches ambíguos.
Aceitamos o match se a distância do vizinho mais próximo for significativamente menor que a do segundo mais próximo ($d_1 < 0.75 \cdot d_2$).

In [ ]:
bf = cv2.BFMatcher(norm)
matches = bf.knnMatch(des1, des2, k=2)

good = []
for m, n in matches:
    # Ratio test
    thresh = 0.75
    # Ajuste para ORB (hammings distances são inteiros, 0.75 as vezes é agressivo demais, mas padrão é ok)
    if m.distance < thresh * n.distance:
        good.append(m)

img_matches = cv2.drawMatches(img1_cl, kp1, img2_cl, kp2, good, None, flags=2)

plt.figure(figsize=(12,6))
plt.imshow(img_matches)
plt.title(f"Matches Filtrados: {len(good)}")
plt.axis('off')
plt.show()

## 3) Matching Geométrico (RANSAC)

O RANSAC (Random Sample Consensus) limpa os outliers restantes impondo uma restrição geométrica (Homografia). Ele encontra a matriz de transformação que alinha a maioria dos pontos.

In [ ]:
if len(good) >= 4:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

    # Encontrar Homografia com RANSAC
    M_hom, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    matchesMask = mask.ravel().tolist()

    # Visualizar apenas os Inliers (Verdes)
    draw_params = dict(matchColor = (0,255,0), 
                       singlePointColor = None,
                       matchesMask = matchesMask, 
                       flags = 2)

    img3 = cv2.drawMatches(img1_cl, kp1, img2_cl, kp2, good, None, **draw_params)

    plt.figure(figsize=(12,6))
    plt.imshow(img3)
    plt.title(f"RANSAC Inliers ({sum(matchesMask)}/{len(good)})")
    plt.axis('off')
    plt.show()
else:
    print("Matches insuficientes para RANSAC.")

## 4) Outros Descritores: SURF, FAST e ORB

- **SURF:** Versão acelerada do SIFT usando Box Filters e Imagens Integrais. (frequentemente patenteado).
- **FAST:** Detector de cantos de alta velocidade (Decision Tree em círculo de pixels). Não descreve o ponto.
- **ORB:** Combinação de FAST (detector) + BRIEF (descritor binário). É a escolha padrão para aplicações em tempo real (SLAM, Mobile).

In [ ]:
# Comparação Rápida com ORB (se já não estiver usando)
orb = cv2.ORB_create()
kp_o, des_o = orb.detectAndCompute(img1_cl, None)

# ORB gera muito mais pontos por padrão (nfeatures=500)
img_orb_vis = cv2.drawKeypoints(img1_cl, kp_o, None, color=(255,0,0))
plt.figure(figsize=(6,6))
plt.imshow(img_orb_vis)
plt.title(f"ORB Keypoints: {len(kp_o)}")
plt.axis('off')
plt.show()

## 5) Representação Global: Bag of Visual Words (BoVW)

Para classificar cenas inteiras usando SIFT, usamos BoVW:
1. Extraímos SIFT de muitas imagens.
2. Clusterizamos (K-Means) para achar "Palavras Visuais" (centróides).
3. Convertemos cada nova imagem em um histograma de palavras.

In [ ]:
from sklearn.cluster import KMeans

# Demo BoVW Simplificado
# Dataset: 4 Imagens
imgs_bovw = [data.chelsea(), data.astronaut(), data.coffee(), data.rocket()]
gray_imgs = [cv2.cvtColor(i, cv2.COLOR_RGB2GRAY) for i in imgs_bovw]
labels = ['Gato', 'Astronauta', 'Café', 'Foguete']

# 1. Extrair Features
all_des = []
des_list = []
for g in gray_imgs:
    kp, des = detector.detectAndCompute(g, None)
    if des is None: des = np.array([])
    all_des.append(des)
    des_list.append(des)

# Empilhar (supondo que achamos features em todas)
if all(d.shape[0] > 0 for d in all_des):
    stack = np.vstack(all_des)

    # 2. Criar Vocabulário (K=5 palavras para demo visual)
    kmeans = KMeans(n_clusters=5, random_state=0, n_init=10)
    kmeans.fit(stack)

    # 3. Gerar Histogramas
    plt.figure(figsize=(15,3))
    for i, (des, label) in enumerate(zip(des_list, labels)):
        words = kmeans.predict(des)
        hist, bin_edges = np.histogram(words, bins=5, range=(0,5))
        
        # Normalizar
        hist = hist / hist.sum()
        
        plt.subplot(1, 4, i+1)
        plt.bar(range(5), hist, color='purple', alpha=0.7)
        plt.title(label)
        plt.ylim(0, 1.0)
        plt.xlabel("Visual Word Index")
    
    plt.suptitle("Assinaturas BoVW")
    plt.show()

## 6) Exercícios

1.  **Robustez:** Teste o matching SIFT/ORB com uma imagem borrada (`cv2.GaussianBlur`). O número de inliers cai drasticamente?
2.  **RANSAC:** Modifique o `ransacReprojThreshold` (o valor 5.0 no código). Se aumentar para 20.0, o que acontece com a precisão do alinhamento?
3.  **BoVW:** Por que usar um vocabulário pequeno (ex: 5 palavras) é ruim para discriminar objetos complexos? Pense em termos de "poder de representação".